# An example pipeline using a (really cruddy) neural network in ```numpy```

In [2]:
# path tools
import sys,os
sys.path.append(os.path.join(".."))

# image processing 
import cv2 

# neural networks with numpy
import numpy as np
from tensorflow.keras.datasets import cifar10
from utils.neuralnetwork import NeuralNetwork

# machine learning tools
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [12]:
X_train.shape

(50000, 32, 32, 3)

In [7]:
labels = ["airplane", 
          "automobile", 
          "bird", 
          "cat", 
          "deer", 
          "dog", 
          "frog", 
          "horse", 
          "ship", 
          "truck"]

### List comprehensions

In [ ]:
# for loop
out_list = []
for x in data:
    out = do_something(x)
    out_list.append(out)
    
# comprehension    
out_list = [do_something(x) for x in data]

### Convert all the data to greyscale

In [14]:
X_train_grey = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in X_train])
X_test_grey = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in X_test])

### Normalize the values

In [16]:
def minmax(data):
    X_norm = (data - data.min())/(data.max() - data.min())
    return X_norm

In [17]:
X_train_scaled = minmax(X_train_grey)
X_test_scaled = minmax(X_test_grey)

### Reshaping the data

In [18]:
nsamples, nx, ny = X_train_scaled.shape
X_train_dataset = X_train_scaled.reshape((nsamples,nx*ny))

In [19]:
nsamples, nx, ny = X_test_scaled.shape
X_test_dataset = X_test_scaled.reshape((nsamples,nx*ny))

## Simple logistic regression classifier

In [23]:
clf = LogisticRegression(penalty='none', 
                         tol=0.1, 
                         solver='saga',
                         multi_class="multinomial").fit(X_train_dataset, y_train)

/home/au564346/cds-visual/cv101/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [24]:
y_pred = clf.predict(X_test_dataset)

In [29]:
report = classification_report(y_test, y_pred, target_names=labels)
print(report)

              precision    recall  f1-score   support

    airplane       0.35      0.35      0.35      1000
  automobile       0.38      0.37      0.38      1000
        bird       0.28      0.19      0.22      1000
         cat       0.23      0.11      0.15      1000
        deer       0.25      0.20      0.22      1000
         dog       0.24      0.40      0.30      1000
        frog       0.29      0.31      0.30      1000
       horse       0.32      0.31      0.31      1000
        ship       0.34      0.44      0.38      1000
       truck       0.41      0.43      0.42      1000

    accuracy                           0.31     10000
   macro avg       0.31      0.31      0.30     10000
weighted avg       0.31      0.31      0.30     10000



## Neural network classifier

In [30]:
y_train = LabelBinarizer().fit_transform(y_train)
y_test = LabelBinarizer().fit_transform(y_test)

In [47]:
print("[INFO] training network...")
input_shape = X_train_dataset.shape[1]
nn = NeuralNetwork([input_shape, 64, 10])
print(f"[INFO] {nn}")
nn.fit(X_train_dataset, y_train, epochs=150, displayUpdate=1)

[INFO] training network...
[INFO] NeuralNetwork: 1024-128-32-10
[INFO] epoch=1, loss=20624.4194879
[INFO] epoch=2, loss=20068.1314784
[INFO] epoch=3, loss=19968.1115003
[INFO] epoch=4, loss=19691.5644965
[INFO] epoch=5, loss=19625.7357842
[INFO] epoch=6, loss=19552.1077400
[INFO] epoch=7, loss=19330.0737219
[INFO] epoch=8, loss=19246.2559796
[INFO] epoch=9, loss=19010.1327551
[INFO] epoch=10, loss=18686.3972820


In [48]:
predictions = nn.predict(X_test_dataset)

In [49]:
y_pred = predictions.argmax(axis=1)
report = classification_report(y_test.argmax(axis=1), y_pred, target_names=labels)

In [50]:
print(report)

              precision    recall  f1-score   support

    airplane       0.68      0.17      0.27      1000
  automobile       0.35      0.67      0.45      1000
        bird       0.37      0.18      0.24      1000
         cat       0.33      0.23      0.27      1000
        deer       0.26      0.47      0.33      1000
         dog       0.48      0.19      0.28      1000
        frog       0.32      0.51      0.39      1000
       horse       0.41      0.51      0.46      1000
        ship       0.54      0.40      0.46      1000
       truck       0.49      0.40      0.44      1000

    accuracy                           0.37     10000
   macro avg       0.42      0.37      0.36     10000
weighted avg       0.42      0.37      0.36     10000

